In [168]:
import numpy as np

In [169]:
f_1_range = (0, 1000)
f_2_range = (0, np.log(1000))

f_1 = 500
f_2 = np.log(500)

In [170]:
denominator = np.divide(
    f_1_range[1] - f_1,
    f_1_range[1] - f_1_range[0]) + \
              np.divide(
                  f_2_range[1] - f_2,
                  f_2_range[1] - f_2_range[0]
              )

num_1 = np.divide(f_1_range[1] - f_1, f_1_range[1] - f_1_range[0])
print(f"{num_1=}")
w_1 = np.divide(
    num_1,
    denominator
)
print(f"{denominator=}")
print(f"{w_1=}")
num_2 = np.divide(f_2_range[1] - f_2, f_2_range[1] - f_2_range[0])
print(f"{num_2=}")
w_2 = np.divide(
    np.divide(f_2_range[1] - f_2, f_2_range[1] - f_2_range[0]),
    denominator
)
print(f"{w_2=}")

num_1=0.5
denominator=0.6003433318879937
w_1=0.8328567561957783
num_2=0.10034333188799374
w_2=0.16714324380422174


In [171]:
np.isclose(w_1 + w_2, 1)

True

In [172]:
F = w_1 * f_1 + w_2 * f_2
F

417.46710785443145

In [173]:
########################

In [174]:
from dataclasses import dataclass

In [187]:
@dataclass
class PartialReward:
    min_value: float
    max_value: float
    reward_value: float

    @property
    def max_diff_min(self):
        return self.max_value - self.min_value

    @property
    def max_diff_actual(self):
        return self.max_value - self.reward_value

    @property
    def ma_d_ac_div_ma_d_mi(self):
        return np.divide(self.max_diff_actual, self.max_diff_min)

@dataclass
class PartialRewardGenerator:
    min_value: float
    max_value: float

    def generate_reward(self, reward_value: float):
        return PartialReward(self.min_value, self.max_value, reward_value)

def chebyshev_weight_computation(*fs: PartialReward):
    numerators = [f.ma_d_ac_div_ma_d_mi for f in fs]
    denominator = np.sum([f.ma_d_ac_div_ma_d_mi for f in fs])

    if denominator != 0:
        return np.divide(numerators, denominator)
    else:
        max_reward = max([f.reward_value for f in fs])
        return [1 if f.reward_value == max_reward else 0 for f in fs]

def chebyshev_reward_computation(*fs: PartialReward):
    weights = chebyshev_weight_computation(*fs)
    assert np.isclose(np.sum(weights), 1)
    return np.sum([w * f.reward_value for w, f in zip(weights, fs)])

In [188]:
rg_1 = PartialRewardGenerator(0, 1000)
f_1 = rg_1.generate_reward(1000)
rg_2 = PartialRewardGenerator(0, np.log(1000))
f_2 = rg_2.generate_reward(np.log(1000))

chebyshev_reward_computation(f_1, f_2)

1000.0